# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from cassandra.query import dict_factory

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment it to get total number of rows 
#print(len(full_data_rows_list))
# uncomment it to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling in Apache Cassandra.

## The CSV file titled <font color=red>event_datafile_new.csv</font> is created.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appear like in the <font color=red>**event_datafile_new.csv**</font>:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
keyspace = session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION =
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

## <font color=navy> Query 1:  Give the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4\</font> 

In [8]:
# create 'query' table music_history_session
query = "CREATE TABLE IF NOT EXISTS music_history_session"
# determine needed columns, assign correct data type to columns and identify the primary key
query = query + "(sessionid int, iteminsession int, artist text, songtitle text, songlength float,\
                PRIMARY KEY ((sessionid, iteminsession)))" #sessionid and iteminsession as partition key
rows = session.execute(query)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# insert initial values into the `query` talbe variables
        query = "INSERT INTO music_history_session (sessionid, iteminsession, artist, songtitle, songlength)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # assign column in csv file into each column in the query table
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
# SELECT needed information from the 'query' table and make output as dataframe
cassandra.query.dict_factory
session.row_factory = dict_factory
rows = session.execute("SELECT artist, songtitle, songlength FROM music_history_session WHERE sessionid=338 and iteminsession=4")
data = list(rows)
df = pd.DataFrame(data)
df

,artist,songlength,songtitle
0,Faithless,495.307312,Music Matters (Mark Knight Dub)


### Query1 result:
![](https://r766469c826649xjupyterlrhraafil.udacity-student-workspaces.com/files/images/query1.png)

## <font color=navy> Query 2: Give the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 </font>

In [11]:
# create 'query' table music_history_user
query = "CREATE TABLE IF NOT EXISTS music_history_user"
# determine needed columns, assign correct data type to columns and identify the primary key
query = query + "(userid int, sessionid int, iteminsession text, artist text, songtitle text, firstname text, lastname text,\
                PRIMARY KEY ((userid, sessionid),iteminsession))" #userid and sessionid as partition key, iteminsession as clustering key for sorting
rows = session.execute(query)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# insert initial values into the `query` talbe variables
        query = "INSERT INTO music_history_user (userid, sessionid, iteminsession, artist, songtitle, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # assign column in csv file into each column in the query table
        session.execute(query, (int(line[10]), int(line[8]), line[3], line[0], line[9], line[1], line[4]))

In [13]:
# SELECT needed information from the 'query' table and make output as dataframe
cassandra.query.dict_factory
session.row_factory = dict_factory
rows = session.execute("SELECT artist, songtitle, firstname, lastname FROM music_history_user WHERE userid=10 and sessionid=182")
data=list(rows)
df=pd.DataFrame(data)
df

,artist,firstname,lastname,songtitle
0,Down To The Bone,Sylvie,Cruz,Keep On Keepin' On
1,Three Drives,Sylvie,Cruz,Greece 2000
2,Sebastien Tellier,Sylvie,Cruz,Kilometer
3,Lonnie Gordon,Sylvie,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...


### Query2 result:
![](https://r766469c826649xjupyterlrhraafil.udacity-student-workspaces.com/files/images/query2.png)

## <font color=navy> Query 3: Give the every user name (first and last) in music app history who listened to the song 'All Hands Against His Own'</font>

In [14]:
# create 'query' table music_history_song
query = "CREATE TABLE IF NOT EXISTS music_history_song"
# determine needed columns, assign correct data type to columns and identify the primary key
query = query + "(songtitle text, userid int, firstname text, lastname text,\
                PRIMARY KEY (songtitle, userid))"  #songtitle and userid as partition key, userid along with songtitle to make records unique
rows = session.execute(query)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# insert initial values into the `query` talbe variables
        query = "INSERT INTO music_history_song (songtitle, userid, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # assign column in csv file into each column in the query table
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
# SELECT needed information from the 'query' table and make output as dataframe
cassandra.query.dict_factory
session.row_factory = dict_factory
rows = session.execute("SELECT firstname, lastname FROM music_history_song WHERE songtitle='All Hands Against His Own'")
data=list(rows)
df=pd.DataFrame(data)
df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Query3 result:
![](https://r766469c826649xjupyterlrhraafil.udacity-student-workspaces.com/files/images/query3.png)

### Drop the tables before closing out the sessions

In [17]:
# Drop the tables created above

query = "drop table music_history_session"
rows = session.execute(query)

query = "drop table music_history_user"
rows = session.execute(query)

query = "drop table music_history_song"
rows = session.execute(query)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()